# Convert the daily netcdf files from JAXA to one zarr file

In [2]:
# Use a dask client
from dask.distributed import Client, LocalCluster, worker, progress, wait
import dask
from dask_jobqueue import SLURMCluster


In [3]:
cluster = SLURMCluster()
client=Client(cluster)
cluster.scale(32)
client

Client Scheduler: tcp://146.118.38.87:35442 Dashboard: http://146.118.38.87:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import zarr



In [5]:
### Input variables
# years = [2016,2017,2018,2019]
# myfile = '/home/mrayson/scratch/HIMAWARI/raw/%d%02d*.nc'
#outpath = '/home/mrayson/scratch/HIMAWARI/raw_zarr'

years = [2020,2021]
myfile = '/home/mrayson/scratch/HIMAWARI/raw_v2/%d%02d*.nc'
outpath = '/home/mrayson/scratch/HIMAWARI/raw_zarr_v2'

In [6]:
%%time

# Convert the dataset to zarr (to this once only)
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)

mode='w'
append_dim=None
encoding = {'sea_surface_temperature': {'compressor': compressor}}

# mode = 'a'
# append_dim='time'
# encoding=None

for year in years:
    for mo in range(1,13):
        
        # Skip a few bad section
        #if year==2017 and mo == 10:
        #    continue
        

        filestr = myfile%(year,mo)
        print(filestr)
        ds = xr.open_mfdataset(sorted(glob.glob(filestr)),
                               combine='by_coords',
                               parallel=True)
        # Mask bad flagged data
        sst = ds['sea_surface_temperature'].where(ds['quality_level']>4, np.nan)
        
        sst=sst.assign_coords({'lon': (((sst.lon ) % 360))})
        
        # Save to zarr format
        dsout = xr.Dataset({'sea_surface_temperature':sst}, attrs=ds.attrs)

        dsout.to_zarr(outpath, encoding=encoding,\
                     mode=mode, append_dim=append_dim)
        
        mode='a'
        append_dim='time'
        encoding=None

        #                       concat_dim='time',
#                       data_vars=['sea_surface_temperature','quality_level'])

/home/mrayson/scratch/HIMAWARI/raw_v2/202001*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202002*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202003*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202004*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202005*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202006*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202007*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202008*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202009*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202010*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202011*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202012*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202101*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202102*.nc
/home/mrayson/scratch/HIMAWARI/raw_v2/202103*.nc


/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/xarray/core/dataset.py:1658: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  append_dim=append_dim,


/home/mrayson/scratch/HIMAWARI/raw_v2/202104*.nc


/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/xarray/core/dataset.py:1658: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  append_dim=append_dim,


/home/mrayson/scratch/HIMAWARI/raw_v2/202105*.nc


/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/xarray/core/dataset.py:1658: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  append_dim=append_dim,


/home/mrayson/scratch/HIMAWARI/raw_v2/202106*.nc


/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/xarray/core/dataset.py:1658: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  append_dim=append_dim,


/home/mrayson/scratch/HIMAWARI/raw_v2/202107*.nc


OSError: no files to open